In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import time

In [21]:
df=pd.read_csv("mnist_test.csv")
df.head()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
X = df.iloc[:, 1:].values.astype(np.float32)
y = df.iloc[:, 0].values.reshape(-1, 1)

encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [25]:
def create_model(input_dim, output_dim):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_dim,)),
        Dense(256, activation='relu'),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

results = []

In [33]:
import warnings
warnings.filterwarnings('ignore')
with tf.device('/CPU:0'):
    model_cpu = create_model(X.shape[1], y_encoded.shape[1])
    start_cpu = time.time()
    model_cpu.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    end_cpu = time.time()
    cpu_time = end_cpu - start_cpu
    _, cpu_acc = model_cpu.evaluate(X_test, y_test, verbose=0)
    results.append(['CPU', cpu_time, cpu_acc])

try:
    model_gpu = create_model(X.shape[1], y_encoded.shape[1])
    start_gpu = time.time()
    model_gpu.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    end_gpu = time.time()
    gpu_time = end_gpu - start_gpu
    _, gpu_acc = model_gpu.evaluate(X_test, y_test, verbose=0)
    results.append(['GPU', gpu_time, gpu_acc])
except RuntimeError as e:
    print("GPU unavailable:", e)
    results.append(['GPU', None, None])

In [35]:
result_df = pd.DataFrame(results, columns=["Device", "Training Time (s)", "Test Accuracy"])
print(result_df)

print("\n⚡ GPU Benefits:\nGPUs can parallelize matrix operations and accelerate training, especially for large models and datasets.")

  Device  Training Time (s)  Test Accuracy
0    CPU           4.846828         0.9340
1    GPU           5.494228         0.9325
2    CPU           4.539693         0.9290
3    GPU           5.029863         0.9255
4    CPU           4.745769         0.9345
5    GPU           4.948165         0.9445

⚡ GPU Benefits:
GPUs can parallelize matrix operations and accelerate training, especially for large models and datasets.
